# Documentation Writing Crew

This notebook demonstrates how to create an AI crew for writing documentation using CrewAI Flows.
The crew will analyze code from any public GitHub repository and generate comprehensive documentation
by working collaboratively using specialized agents with different roles and responsibilities.
CrewAI Flows enable coordinated execution and communication between agents to produce high-quality
documentation for any codebase.

### Initialization and Setup
Initial imports for the CrewAI Flow and Crew and setting up the environment

In [1]:
# Importing necessary libraries
import os
import yaml
import subprocess
from pathlib import Path
from pydantic import BaseModel

# Importing Crew related components
from crewai import Agent, Task, Crew

# Importing CrewAI Flow related components
from crewai.flow.flow import Flow, listen, start

# Setting up the environment
os.environ["NVIDIA_NIM_API_KEY"] = "nvapi-xxxxxxx"

In [2]:
project_url = "https://github.com/joaomdmoura/gioco"

## Plan for our Flow

1. Clone the repository for the project
2. Plan the documentation for the project **[Crew of Agents]** 
3. Create the documentation for the project **[Crew of Agents]**

# ![CrewAIFlow.png](CrewAIFlow.png)

# Create Planning Crew

Initial strucutre data we will use to capture the output of the planning crew

In [3]:
# Define data structures to capture documentation planning output
class CodeComponent(BaseModel):
    """Represents a key code component"""
    name: str
    usage: str
    integration: str

class DocItem(BaseModel):
    """Represents a documentation item"""
    title: str
    description: str
    prerequisites: str
    examples: list[str]
    goal: str

class DocPlan(BaseModel):
    """Documentation plan"""
    overview: str
    components: list[CodeComponent]
    docs: list[DocItem]

In [4]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
)

# Load agent and task configurations from YAML files
with open('config/planner_agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/planner_tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

code_explorer = Agent(config=agents_config['code_explorer'], tools=[
    DirectoryReadTool(),
    FileReadTool()
])
documentation_planner = Agent(config=agents_config['documentation_planner'], tools=[
    DirectoryReadTool(),
    FileReadTool()
])

analyze_codebase = Task(
  config=tasks_config['analyze_codebase'],
  agent=code_explorer
)
create_documentation_plan = Task(
  config=tasks_config['create_documentation_plan'],
  agent=documentation_planner,
  output_pydantic=DocPlan
)

planning_crew = Crew(
    agents=[code_explorer, documentation_planner],
    tasks=[analyze_codebase, create_documentation_plan],
    verbose=True
)

/Users/joaomoura/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create Documentation Crew

Crew of AI Agents to execute the documentation plan and create the documentation

In [5]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
)

# Load agent and task configurations from YAML files
with open('config/documentation_agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/documentation_tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

overview_writer = Agent(config=agents_config['overview_writer'], tools=[
    DirectoryReadTool(),
    FileReadTool()
])

documentation_reviewer = Agent(config=agents_config['documentation_reviewer'], tools=[
    DirectoryReadTool(directory="docs/"),
    FileReadTool()
])

draft_documentation = Task(
  config=tasks_config['draft_documentation'],
  agent=overview_writer
)

qa_review_documentation = Task(
  config=tasks_config['qa_review_documentation'],
  agent=documentation_reviewer
)

documentation_crew = Crew(
    agents=[overview_writer, documentation_reviewer],
    tasks=[draft_documentation, qa_review_documentation],
    verbose=True
)

2024-12-14 06:31:50,540 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


# Create Documentation Flow

A Flow to create the documentation for the project where we will use the planning crew to plan the documentation and the documentation crew to create the documentation

In [6]:
class DocumentationState(BaseModel):
  """
  State for the documentation flow
  """
  project_url: str = project_url
  repo_path: Path = "workdir/"

class CreateDocumentationFlow(Flow[DocumentationState]):
  # Clone the repository, initial step
  # No need for AI Agents on this step, so we just use regular Python code
  @start()
  def clone_repo(self):
    print(f"# Cloning repository: {self.state.project_url}")
    # Extract repo name from URL
    repo_name = self.state.project_url.split("/")[-1]
    self.state.repo_path = f"{self.state.repo_path}{repo_name}"

    # Clone the repository
    subprocess.run(["git", "clone", self.state.project_url, self.state.repo_path])
    return self.state

  @listen(clone_repo)
  def plan_docs(self):
    result = planning_crew.kickoff(inputs={'repo_path': self.state.repo_path})
    return result

  @listen(plan_docs)
  def create_docs(self, plan):
    for doc in plan.pydantic.docs:
      result = documentation_crew.kickoff(inputs={
        'repo_path': self.state.repo_path,
        'title': doc.title,
        'overview': plan.pydantic.overview,
        'components': [c.dict() for c in plan.pydantic.components],
        'description': doc.description,
        'prerequisites': doc.prerequisites,
        'examples': doc.examples,
        'goal': doc.goal
      })

      # Save documentation to file in docs folder
      docs_dir = Path("docs")
      docs_dir.mkdir(exist_ok=True)
      title = doc.title.lower().replace(" ", "_") + ".mdx"
      with open(docs_dir / title, "w") as f:
          f.write(result.raw)


Implementing helper methods to plot and execute the flow in a Jupyter notebook

In [7]:
def kickoff():
  import nest_asyncio
  import asyncio

  # Apply a patch to allow nested asyncio loops in Jupyter
  nest_asyncio.apply()

  flow = CreateDocumentationFlow()

  # Get or create an event loop and run the flow asynchronously
  loop = asyncio.get_event_loop()
  result = loop.run_until_complete(flow.kickoff_async())
  return result

def plot():
  flow = CreateDocumentationFlow()
  flow.plot()

In [8]:
# Plot the flow
plot()

# Display the flow visualization using IFrame
from IPython.display import IFrame

# Display the flow visualization
IFrame(src='./crewai_flow.html', width='100%', height=600)

Plot saved as crewai_flow.html


In [9]:
kickoff()

fatal: destination path 'workdir/gioco' already exists and is not an empty directory.


# Cloning repository: https://github.com/joaomdmoura/gioco
# Agent: Code Explorer
## Task: Analyze the codebase at workdir/gioco to create a developer-focused technical overview:
1. Map out the core architecture and components by examining file structure and imports 2. Identify key classes, functions and their interactions through static code analysis 3. Document APIs and interfaces with usage examples 4. Analyze and diagram data/control flows between components 5. Note implemented design patterns and their practical applications 6. Identify common usage patterns and integration points
Focus on details that would help another engineer understand and work with the codebase.



# Agent: Code Explorer
## Thought: To begin analyzing the codebase at workdir/gioco, I should first understand the overall structure of the project. This involves looking at the directory structure and identifying key files and folders.
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"work